# 상품 리뷰의 호불호 예측하기 (Predicting sentiment from product reviews)

## Import Graphlab

graphlab을 import합니다. 에러가 나는 경우 'install graphlab' 노트북을 열어서 설치해야 합니다.

In [1]:
import graphlab

## 데이터 화일 업로드하기

먼저 amazon_baby.gl.zip 화일을 업로드합니다. Destination Folder에 /library/data 폴더를 지정합니다.

ls 명령을 이용해서 data 폴더 안에 amazon_baby.gl.zip 화일이 제대로 업로드되었는지 확인합시다.

In [2]:
ls ../data

amazon_baby.gl/  home_data.gl/  home_data.gl.zip


이제 unzip 명령을 이용해서 압축화일을 해제합시다.

In [3]:
!unzip -o ../amazon_baby.gl.zip -d ../data

Archive:  ../amazon_baby.gl.zip
  inflating: ../data/amazon_baby.gl/dir_archive.ini  
  inflating: ../data/amazon_baby.gl/m_bfaa91c17752f745.0000  
  inflating: ../data/amazon_baby.gl/m_bfaa91c17752f745.frame_idx  
  inflating: ../data/amazon_baby.gl/m_bfaa91c17752f745.sidx  
 extracting: ../data/amazon_baby.gl/objects.bin  


## 상품 리뷰 데이터를 읽어 들입니다.

아마존에서 판매하는 유아 관련 상품들의 리뷰를 읽어 들입니다.

SFrame 데이터를 읽어 와서 **products**라는 변수에 저장합니다.
**'../data/amazon_baby.gl/'**이라는 폴더를 지정해야 합니다.

    graphlab.SFrame(폴더위치)

In [4]:
products = graphlab.SFrame('../data/amazon_baby.gl')

This non-commercial license of GraphLab Create for academic use is assigned to cutty724@hanmail.net and will expire on December 24, 2018.


[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: /tmp/graphlab_server_1516032667.log


## 데이터를 살펴 봅시다.

먼저 head 메소드를 사용해서 products에 어떤 데이터가 있는지 확인해 봅시다.

In [5]:
products.head()

name,review,rating
Planetwise Flannel Wipes,"These flannel wipes areOK, but in my opinion ...",3.0
Planetwise Wipe Pouch,it came early and was notdisappointed. i love ...,5.0
Annas Dream Full Quiltwith 2 Shams ...,Very soft and comfortableand warmer than it ...,5.0
Stop Pacifier Suckingwithout tears with ...,This is a product wellworth the purchase. I ...,5.0
Stop Pacifier Suckingwithout tears with ...,All of my kids have criednon-stop when I tried to ...,5.0
Stop Pacifier Suckingwithout tears with ...,"When the Binky Fairy cameto our house, we didn't ...",5.0
A Tale of Baby's Dayswith Peter Rabbit ...,"Lovely book, it's boundtightly so you may no ...",4.0
"Baby Tracker&reg; - DailyChildcare Journal, ...",Perfect for new parents.We were able to keep ...,5.0
"Baby Tracker&reg; - DailyChildcare Journal, ...",A friend of mine pinnedthis product on Pinte ...,5.0
"Baby Tracker&reg; - DailyChildcare Journal, ...",This has been an easy wayfor my nanny to record ...,4.0


상품 이름, 리뷰의 텍스트, 리뷰의 점수 등이 포함되어 있습니다.

## 각 리뷰의 단어 개수 벡터를 만들어 봅시다.

먼저 products에서 review 컬럼만을 출력해 봅시다. 지난 시간에 배운 **filtering**을 이용합시다.

In [6]:
products['review']

dtype: str
Rows: 183531
['These flannel wipes are OK, but in my opinion not worth keeping.  I also ordered someImse Vimse Cloth Wipes-Ocean Blue-12 countwhich are larger, had a nicer, softer texture and just seemed higher quality.  I use cloth wipes for hands and faces and have been usingThirsties 6 Pack Fab Wipes, Boyfor about 8 months now and need to replace them because they are starting to get rough and have had stink issues for a while that stripping no longer handles.', 'it came early and was not disappointed. i love planet wise bags and now my wipe holder. it keps my osocozy wipes moist and does not leak. highly recommend it.', 'Very soft and comfortable and warmer than it looks...fit the full size bed perfectly...would recommend to anyone looking for this type of quilt', 'This is a product well worth the purchase.  I have not found anything else like this, and it is a positive, ingenious approach to losing the binky.  What I love most about this product is how much ownership my

type함수를 이용해서 리뷰 컬럼의 유형을 확인해 봅시다.

In [7]:
type(products['review'])

graphlab.data_structures.sarray.SArray

이제 graphlab.text_analytics.count_words라는 메소드를 사용해서 각 review의 단어 개수 벡터를 만든 다음, **word_count**라는 컬럼으로 products에 저장해 봅시다.

    graphlab.text_analytics.count_words(SArray)

주의: 이 메소드는 SFrame이 아니라 SArray를 입력으로 받습니다.

In [8]:
products['word_count'] = graphlab.text_analytics.count_words(products['review'])

자, 이제 다시 head 메소드를 이용해서 word_count 컬럼에 단어 개수 벡터가 추가된 것을 확인해 봅시다.

In [9]:
products['word_count'].head()

dtype: dict
Rows: 10
[{'and': 5, '6': 1, 'stink': 1, 'because': 1, 'ordered': 1, 'just': 1, 'boyfor': 1, 'wipes-ocean': 1, 'wipes,': 1, 'replace': 1, 'not': 1, 'softer': 1, 'are': 3, 'have': 2, 'in': 1, 'need': 1, 'rough': 1, 'ok,': 1, 'issues': 1, 'seemed': 1, 'use': 1, 'blue-12': 1, 'vimse': 1, 'for': 2, 'no': 1, 'that': 1, 'larger,': 1, 'been': 1, 'to': 2, 'someimse': 1, 'quality.': 1, '8': 1, 'flannel': 1, 'worth': 1, 'higher': 1, 'them': 1, 'get': 1, 'keeping.': 1, 'countwhich': 1, 'texture': 1, 'but': 1, 'cloth': 2, 'nicer,': 1, 'they': 1, 'hands': 1, 'fab': 1, 'now': 1, 'had': 2, 'a': 2, 'also': 1, 'about': 1, 'usingthirsties': 1, 'longer': 1, 'i': 2, 'my': 1, 'months': 1, 'wipes': 2, 'these': 1, 'while': 1, 'stripping': 1, 'faces': 1, 'handles.': 1, 'opinion': 1, 'starting': 1, 'pack': 1}, {'and': 3, 'love': 1, 'it': 2, 'highly': 1, 'osocozy': 1, 'bags': 1, 'holder.': 1, 'moist': 1, 'does': 1, 'recommend': 1, 'was': 1, 'wipes': 1, 'it.': 1, 'early': 1, 'disappointed.': 1, 'not'

**word_count** 컬럼을 출력해서 어떤 내용이 들어가 있는지 확인해 봅시다.

In [10]:
products['word_count']

dtype: dict
Rows: 183531
[{'and': 5, '6': 1, 'stink': 1, 'because': 1, 'ordered': 1, 'just': 1, 'boyfor': 1, 'wipes-ocean': 1, 'wipes,': 1, 'replace': 1, 'not': 1, 'softer': 1, 'are': 3, 'have': 2, 'in': 1, 'need': 1, 'rough': 1, 'ok,': 1, 'issues': 1, 'seemed': 1, 'use': 1, 'blue-12': 1, 'vimse': 1, 'for': 2, 'no': 1, 'that': 1, 'larger,': 1, 'been': 1, 'to': 2, 'someimse': 1, 'quality.': 1, '8': 1, 'flannel': 1, 'worth': 1, 'higher': 1, 'them': 1, 'get': 1, 'keeping.': 1, 'countwhich': 1, 'texture': 1, 'but': 1, 'cloth': 2, 'nicer,': 1, 'they': 1, 'hands': 1, 'fab': 1, 'now': 1, 'had': 2, 'a': 2, 'also': 1, 'about': 1, 'usingthirsties': 1, 'longer': 1, 'i': 2, 'my': 1, 'months': 1, 'wipes': 2, 'these': 1, 'while': 1, 'stripping': 1, 'faces': 1, 'handles.': 1, 'opinion': 1, 'starting': 1, 'pack': 1}, {'and': 3, 'love': 1, 'it': 2, 'highly': 1, 'osocozy': 1, 'bags': 1, 'holder.': 1, 'moist': 1, 'does': 1, 'recommend': 1, 'was': 1, 'wipes': 1, 'it.': 1, 'early': 1, 'disappointed.': 1, '

word_count안에 단어를 키로, 단어가 포함된 횟수를 값으로 하는 dictionary 형태의 값이 들어 있다는 것을 확인할 수 있습니다.

graphlab에서 제공하는 분석 기능을 이용해서 간단한 분석을 해 보겠습니다. 먼저 아래 명령어를 실행해서 그래프가 이 노트북에 출력되도록 합시다.

In [11]:
graphlab.canvas.set_target('ipynb')

가장 리뷰가 많은 상품을 찾아 봅시다. 여기서는 SFrame에서 제공하는 편리한 메소드인 SArray.show를 이용하겠습니다. products의 이름 컬럼에 show 메소드를 적용해 봅시다.

    SArray.show()

In [12]:
products['name'].show()

products.show 메소드를 이용해서 위의 출력값과 비교해 봅시다. 언제 어떤 방법을 쓰는 것이 더 좋을까요?

In [13]:
products.show()

## 가장 리뷰가 많은 상품의 리뷰를 살펴 보기

### SFrame 필터링을 배워 봅시다. 

SFrame 필터링은 SFrame에서 특정 조건을 만족하는 데이터만을 추출하는 것을 말합니다. 먼저 SArray를 이용해서 조건문을 만듭니다. 이 결과는 역시 SArray로 저장됩니다.

    ex) SArray > 3 

예를 들어서 products 중에 rating이 3.0 보다 큰 products만을 남기고자 한다면 아래와 같이 조건문을 만들면 됩니다.

    products['rating'] > 3.

products의 각 줄에 대해서 위의 조건문을 실행하고 1 또는 0 (참 또는 거짓)으로 판별한 후 SArray에 저장됩니다. 여기서 3 뒤의 소수점은 float값을 이용하기 위해서 사용합니다.

위의 명령을 실행해서 어떤값이 나오는지 확인해 봅시다.

In [14]:
products['rating'] > 3.

dtype: int
Rows: 183531
[0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, ... ]

이렇게 구해진 SArray를 SFrame에 다시 적용하면 해당 조건을 만족하는 데이터만으로 구성된 SFrame을 얻을 수 있습니다.

    SFrame[SArray]

위의 예에서 rating이 3.0보다 큰 products들은 아래와 같이 실행하면 됩니다.

    products[products['rating'] > 3.0]
    
실행해 봅시다.

In [15]:
products[products['rating']>3.0]

name,review,rating,word_count
Planetwise Wipe Pouch,it came early and was notdisappointed. i love ...,5.0,"{'and': 3, 'love': 1,'it': 2, 'highly': 1, ..."
Annas Dream Full Quiltwith 2 Shams ...,Very soft and comfortableand warmer than it ...,5.0,"{'and': 2, 'quilt': 1,'it': 1, 'comfortable': ..."
Stop Pacifier Suckingwithout tears with ...,This is a product wellworth the purchase. I ...,5.0,"{'ingenious': 1, 'and':3, 'love': 2, ..."
Stop Pacifier Suckingwithout tears with ...,All of my kids have criednon-stop when I tried to ...,5.0,"{'and': 2, 'parents!!':1, 'all': 2, 'puppet.': ..."
Stop Pacifier Suckingwithout tears with ...,"When the Binky Fairy cameto our house, we didn't ...",5.0,"{'and': 2, 'this': 2,'her': 1, 'help': 2, ..."
A Tale of Baby's Dayswith Peter Rabbit ...,"Lovely book, it's boundtightly so you may no ...",4.0,"{'shop': 1, 'noble': 1,'is': 1, 'it': 1, 'as': ..."
"Baby Tracker&reg; - DailyChildcare Journal, ...",Perfect for new parents.We were able to keep ...,5.0,"{'and': 2, 'all': 1,'right': 1, 'when': 1, ..."
"Baby Tracker&reg; - DailyChildcare Journal, ...",A friend of mine pinnedthis product on Pinte ...,5.0,"{'and': 1, 'help': 1,'give': 1, 'is': 1, ' ..."
"Baby Tracker&reg; - DailyChildcare Journal, ...",This has been an easy wayfor my nanny to record ...,4.0,"{'journal.': 1, 'nanny':1, 'standarad': 1, ..."
"Baby Tracker&reg; - DailyChildcare Journal, ...",I love this journal andour nanny uses it ...,4.0,"{'all': 1, 'forget': 1,'just': 1, 'food': 1, ..."


### giraffe_reviews 만들기

자, 이제 가장 리뷰가 많은 상품만 골라서 giraffe_reviews라는 변수에 저장해 봅시다.

위의 예제와 동일한 순서에 따라 먼저 조건문을 만들어 보세요. 먼저 가장 리뷰가 많은 상품의 이름을 확인하세요. 그런 다음 해당 이름과 동일한지를 확인하는 조건문을 만들어 봅시다.

참고: 파이썬에서 동일한 값인지를 평가하는 연산자(operator)는 **==** 입니다.

In [16]:
products['name'] == 'Vulli Sophie the Giraffe Teether'

dtype: int
Rows: 183531
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ... ]

0 또는 1로 이루어진 SArray가 출력되었나요?

다시 한번 type함수를 이용해서 SArray인지 확인해 봅시다.

In [17]:
type(products['name'] == 'Vulli Sophie the Giraffe Teether')

graphlab.data_structures.sarray.SArray

이렇게 구한 SArray를 products에 적용해서 해당 상품만 선택해 봅시다.

In [18]:
products[products['name'] == 'Vulli Sophie the Giraffe Teether']

name,review,rating,word_count
Vulli Sophie the GiraffeTeether ...,He likes chewing on allthe parts especially the ...,5.0,"{'and': 1, 'all': 1,'because': 1, 'it': 1, ..."
Vulli Sophie the GiraffeTeether ...,My son loves this toy andfits great in the diaper ...,5.0,"{'and': 1, 'right': 1,'help': 1, 'just': 1, ..."
Vulli Sophie the GiraffeTeether ...,There really should be alarge warning on the ...,1.0,"{'and': 2, 'all': 1,'would': 1, 'latex.': 1, ..."
Vulli Sophie the GiraffeTeether ...,All the moms in my moms'group got Sophie for ...,5.0,"{'and': 2, 'one!': 1,'all': 1, 'love': 1, ..."
Vulli Sophie the GiraffeTeether ...,I was a little skepticalon whether Sophie was ...,5.0,"{'and': 3, 'all': 1,'months': 1, 'old': 1, ..."
Vulli Sophie the GiraffeTeether ...,I have been reading aboutSophie and was going ...,5.0,"{'and': 6, 'seven': 1,'already': 1, 'love': 1, ..."
Vulli Sophie the GiraffeTeether ...,My neice loves her sophieand has spent hours ...,5.0,"{'and': 4, 'drooling,':1, 'love': 1, ..."
Vulli Sophie the GiraffeTeether ...,What a friendly face!And those mesmerizing ...,5.0,"{'and': 3, 'chew': 1,'be': 1, 'is': 1, ..."
Vulli Sophie the GiraffeTeether ...,We got this just for myson to chew on instea ...,5.0,"{'chew': 2, 'seemed': 1,'because': 1, 'about.': ..."
Vulli Sophie the GiraffeTeether ...,"My baby seems to likethis toy, but I could ...",3.0,"{'and': 2, 'already': 1,'some': 1, 'it': 3, ..."


원하는 결과값이 나왔다면 이 결과를 **giraffe_reviews**라는 변수에 저장합시다.

In [19]:
giraffe_reviews = products[products['name'] == 'Vulli Sophie the Giraffe Teether']

len 함수를 이용해서 giraffe_reviews의 개수를 구해 봅시다.

In [20]:
len(giraffe_reviews)

785

**785**개가 나왔나요?

이제 giraffe_reviews 데이터 중에 rating의 분포를 확인해 봅시다. 위에서 사용했던 show 메소드를 이용하면 됩니다.

In [21]:
giraffe_reviews['rating'].show()

show 메소드에서 view유형이 지정되지 않은 경우, graphlab이 숫자값에 대해서는 분포 그래프를, 문자열에 대해서는 빈도 그래프를 자동으로 그려 줍니다.

rating의 경우 값은 숫자지만 분포 그래프보다는 빈도 그래프가 더 유용하기 때문에 빈도 그래프를 그려 보겠습니다.

    SArray.show(view='Categorical')

giraffe_reviews 데이터의 빈도 그래프를 그려 봅시다.

In [22]:
giraffe_reviews['rating'].show(view='Categorical')

## 호불호 구분자 만들기

이번에는 products 전체의 rating 데이터를 살펴 봅시다.

In [23]:
products['rating'].show(view='Categorical')

### 긍정적인 리뷰와 부정적인 리뷰 정의하기

중립적인 태도를 보이는 리뷰(rating이 3인 리뷰)는 무시하고 ** rating이 4이상인 리뷰들은 긍정적인 리뷰로, 2이하인 리뷰들은 부정적인 리뷰**로 구분하고자 합니다. 

왜 중립적인 태도를 보이는 리뷰를 무시할까요? (참고: 우리는 모델을 학습시키려고 합니다.)

#### 모든 3스타 리뷰 제거하기

위에서 배운 필터링을 이용해서 products에서 rating이 3인 리뷰를 제외해 봅시다.

참고: 파이썬에서 값이 같지 않은지를 평가하는 연산자(operator)는 **!=** 입니다.

In [24]:
products[products['rating']!=3]

name,review,rating,word_count
Planetwise Wipe Pouch,it came early and was notdisappointed. i love ...,5.0,"{'and': 3, 'love': 1,'it': 2, 'highly': 1, ..."
Annas Dream Full Quiltwith 2 Shams ...,Very soft and comfortableand warmer than it ...,5.0,"{'and': 2, 'quilt': 1,'it': 1, 'comfortable': ..."
Stop Pacifier Suckingwithout tears with ...,This is a product wellworth the purchase. I ...,5.0,"{'ingenious': 1, 'and':3, 'love': 2, ..."
Stop Pacifier Suckingwithout tears with ...,All of my kids have criednon-stop when I tried to ...,5.0,"{'and': 2, 'parents!!':1, 'all': 2, 'puppet.': ..."
Stop Pacifier Suckingwithout tears with ...,"When the Binky Fairy cameto our house, we didn't ...",5.0,"{'and': 2, 'this': 2,'her': 1, 'help': 2, ..."
A Tale of Baby's Dayswith Peter Rabbit ...,"Lovely book, it's boundtightly so you may no ...",4.0,"{'shop': 1, 'noble': 1,'is': 1, 'it': 1, 'as': ..."
"Baby Tracker&reg; - DailyChildcare Journal, ...",Perfect for new parents.We were able to keep ...,5.0,"{'and': 2, 'all': 1,'right': 1, 'when': 1, ..."
"Baby Tracker&reg; - DailyChildcare Journal, ...",A friend of mine pinnedthis product on Pinte ...,5.0,"{'and': 1, 'help': 1,'give': 1, 'is': 1, ' ..."
"Baby Tracker&reg; - DailyChildcare Journal, ...",This has been an easy wayfor my nanny to record ...,4.0,"{'journal.': 1, 'nanny':1, 'standarad': 1, ..."
"Baby Tracker&reg; - DailyChildcare Journal, ...",I love this journal andour nanny uses it ...,4.0,"{'all': 1, 'forget': 1,'just': 1, 'food': 1, ..."


위의 결과값을 **filtered_products**에 저장하세요.

참고: 코세라에서 제공하는 노트북에서는 필터링한 데이터를 products에 다시 저장합니다. 이 방법은 데이터가 크고 메모리가 부족한 상황에서는 유용할 수 있지만 디버깅을 하기 어려워지고 학습 시에 헷갈릴 위험이 있습니다. 따라서 여기에서는 filtered_products라는 변수명을 사용해서 별도로 저장하고자 합니다.

In [25]:
filtered_products = products[products['rating']!=3]
print filtered_products

+-------------------------------+-------------------------------+--------+
|              name             |             review            | rating |
+-------------------------------+-------------------------------+--------+
|     Planetwise Wipe Pouch     | it came early and was not ... |  5.0   |
| Annas Dream Full Quilt wit... | Very soft and comfortable ... |  5.0   |
| Stop Pacifier Sucking with... | This is a product well wor... |  5.0   |
| Stop Pacifier Sucking with... | All of my kids have cried ... |  5.0   |
| Stop Pacifier Sucking with... | When the Binky Fairy came ... |  5.0   |
| A Tale of Baby's Days with... | Lovely book, it's bound ti... |  4.0   |
| Baby Tracker&reg; - Daily ... | Perfect for new parents. W... |  5.0   |
| Baby Tracker&reg; - Daily ... | A friend of mine pinned th... |  5.0   |
| Baby Tracker&reg; - Daily ... | This has been an easy way ... |  4.0   |
| Baby Tracker&reg; - Daily ... | I love this journal and ou... |  4.0   |
+------------------------

### sentiment 구하기

**데이터를 이용해서 classifier를 학습을 시키기 위해서는 sentiment (호 또는 불호)로 이루어진 데이터가 필요합니다.**

이제 sentiment (호불호)를 구해서 sentiment라는 컬럼에 저장해 봅시다.

위에서 만든 정의에 따라서 긍정적인 리뷰는 1, 부정적인 리뷰는 0이 나오도록 조건문을 만들어 봅시다. 어떻게 하면 될까요?

In [26]:
filtered_products['rating'] > 3  #컴퓨터는 Y/N로 저장하지 않음. BInary로 1,0으로 저장

dtype: int
Rows: ?
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ... ]

이렇게 구해진 sentiment를 filtered_products의 **sentiment**라는 컬럼에 저장해 주세요.

In [27]:
filtered_products['sentiment'] = filtered_products['rating'] > 3

자, filtered_products를 출력해서 sentiment값이 제대로 저장되었는지 확인해 봅시다.

In [28]:
filtered_products['sentiment'].show(view='Categorical')

### 호불호 구분자 학습시키기

지난 시간에 배운 방법을 이용해서 filtered_products를 **train_data**와 **test_data**를 80:20의 비율로 나눠 봅시다. 여기서 **시드값은 0**을 사용합니다.

    SFrame.random_split(비율, seed=시드값)

In [29]:
train_data, test_data = filtered_products.random_split(0.8, seed=0)

구분자 모델을 생성하는 메소드는 다음과 같습니다.

    graphlab.logistic_classifier.create(학습데이터, target=예측하고자하는값의컬럼이름, features=feature로사용하고자하는데이터의컬럼이름리스트, validation_set=검증데이터)

지난 시간과 다르게 이번에는 검증데이터에 None이 아닌 test_data를 사용합시다.

주의: 예측하고자 하는 값의 컬럼 이름, feature로 사용하고자 하는 데이터의 컬럼 이름 등은 학습 데이터 안에 반드시 존재해야 합니다.

이렇게 생성한 모델을 **sentiment_model**이라는 변수에 저장하세요.

In [30]:
sentiment_model = graphlab.logistic_classifier.create(train_data, target='sentiment', features=['word_count'], validation_set=test_data)

WARNING: The number of feature dimensions in this problem is very large in comparison with the number of examples. Unless an appropriate regularization value is set, this model may not provide accurate predictions for a validation/test set.

Logistic regression:

--------------------------------------------------------

Number of examples          : 133448

Number of classes           : 2

...

| 9         | 17       | 1.000000  | 12.808930    | 0.987066          | 0.919709            |

| 10        | 18       | 1.000000  | 13.788271    | 0.988715          | 0.916256            |

+-----------+----------+-----------+--------------+-------------------+---------------------+

TERMINATED: Iteration limit reached.

This model may not be optimal. To improve it, consider increasing `max_iterations`.

## 호불호 모델 평가하기

모델이 성공적으로 생성되었나요?

evaluate 메소드를 이용하면 데이터를 평가할 수 있습니다.

    model.evaluate(데이터, metric=메트릭종류)
    
여기서는 **'roc_curve'**라는 메트릭을 사용하고자 합니다.

이제 evaluate 메소드를 이용해서 모델을 평가해 봅시다.

In [31]:
sentiment_model.evaluate(test_data, metric='roc_curve')

{'roc_curve': Columns:
 	threshold	float
 	fpr	float
 	tpr	float
 	p	int
 	n	int
 
 Rows: 100001
 
 Data:
 +-----------+----------------+----------------+-------+------+
 | threshold |      fpr       |      tpr       |   p   |  n   |
 +-----------+----------------+----------------+-------+------+
 |    0.0    |      1.0       |      1.0       | 27976 | 5328 |
 |   1e-05   | 0.909346846847 | 0.998856162425 | 27976 | 5328 |
 |   2e-05   | 0.896021021021 | 0.998748927652 | 27976 | 5328 |
 |   3e-05   | 0.886448948949 | 0.998462968259 | 27976 | 5328 |
 |   4e-05   | 0.879692192192 | 0.998284243637 | 27976 | 5328 |
 |   5e-05   | 0.875187687688 | 0.998212753789 | 27976 | 5328 |
 |   6e-05   | 0.872184684685 | 0.998177008865 | 27976 | 5328 |
 |   7e-05   | 0.868618618619 | 0.998034029168 | 27976 | 5328 |
 |   8e-05   | 0.864677177177 | 0.997998284244 | 27976 | 5328 |
 |   9e-05   | 0.860735735736 | 0.997962539319 | 27976 | 5328 |
 +-----------+----------------+----------------+-------+------

show 메소드를 이용하면 모델을 다양한 뷰로 표시할 수 있습니다.

    model.show(view=뷰유형)
    
먼저 뷰유형을 지정하지 않고 출력해 봅시다.

In [32]:
sentiment_model.show()

그 다음에는 **'Evaluation'**이라는 뷰를 사용해 봅시다.

In [33]:
sentiment_model.show(view='Evaluation')

이 그래프는 수업 중에 배운 ROC 커브와 동일한 그래프입니다. 

## 학습된 모델을 이용해서 Giraffe 리뷰의 호불호를 예측해 보자.

predict 메소드를 이용해서 Giraffe 리뷰의 호불호를 예측해 봅시다.

    model.predict(데이터, output_type=결과유형)
    
결과 유형은 **'probability'**로 지정해 주세요. 단순히 참/거짓이 아니라 확률값을 얻을 수 있습니다.

예측한 결과값은 giraffe_reviews에 **'predicted_sentiment'**라는 컬럼을 만들어 저장하세요.

In [34]:
giraffe_reviews['predicted_sentiment'] = sentiment_model.predict(giraffe_reviews, output_type='probability')

head 메소드로 predicted_sentiment가 잘 생성되었는지 확인해 봅시다.

In [35]:
giraffe_reviews.head()

name,review,rating,word_count,predicted_sentiment
Vulli Sophie the GiraffeTeether ...,He likes chewing on allthe parts especially the ...,5.0,"{'and': 1, 'all': 1,'because': 1, 'it': 1, ...",0.999513023521
Vulli Sophie the GiraffeTeether ...,My son loves this toy andfits great in the diaper ...,5.0,"{'and': 1, 'right': 1,'help': 1, 'just': 1, ...",0.999320678306
Vulli Sophie the GiraffeTeether ...,There really should be alarge warning on the ...,1.0,"{'and': 2, 'all': 1,'would': 1, 'latex.': 1, ...",0.013558811687
Vulli Sophie the GiraffeTeether ...,All the moms in my moms'group got Sophie for ...,5.0,"{'and': 2, 'one!': 1,'all': 1, 'love': 1, ...",0.995769474148
Vulli Sophie the GiraffeTeether ...,I was a little skepticalon whether Sophie was ...,5.0,"{'and': 3, 'all': 1,'months': 1, 'old': 1, ...",0.662374415673
Vulli Sophie the GiraffeTeether ...,I have been reading aboutSophie and was going ...,5.0,"{'and': 6, 'seven': 1,'already': 1, 'love': 1, ...",0.999997148186
Vulli Sophie the GiraffeTeether ...,My neice loves her sophieand has spent hours ...,5.0,"{'and': 4, 'drooling,':1, 'love': 1, ...",0.989190989536
Vulli Sophie the GiraffeTeether ...,What a friendly face!And those mesmerizing ...,5.0,"{'and': 3, 'chew': 1,'be': 1, 'is': 1, ...",0.999563518413
Vulli Sophie the GiraffeTeether ...,We got this just for myson to chew on instea ...,5.0,"{'chew': 2, 'seemed': 1,'because': 1, 'about.': ...",0.970160542725
Vulli Sophie the GiraffeTeether ...,"My baby seems to likethis toy, but I could ...",3.0,"{'and': 2, 'already': 1,'some': 1, 'it': 3, ...",0.195367644588


여기서 predicted_sentiment가 1에 가깝다면 어떤 의미인가요? 또는 0에 가깝다면 어떤 의미인가요?

### 예측한 호불호를 기반으로 리뷰를 소팅해 보자.

분석을 위해서 리뷰를 **내림차순**으로 소팅해 봅시다.

    SFrame.sort(소팅할컬럼이름, ascending=True/False)
    
이 결과를 **giraffe_reviews**에 저장합니다.

참고: sorting 값은 굳이 새로운 변수에 저장할 필요가 없습니다.

In [36]:
giraffe_reviews = giraffe_reviews.sort('predicted_sentiment', ascending=False)

giraffe_reviews의 데이터를 살펴 봅시다.

In [37]:
giraffe_reviews

name,review,rating,word_count,predicted_sentiment
Vulli Sophie the GiraffeTeether ...,"Sophie, oh Sophie, yourtime has come. My ...",5.0,"{'giggles': 1, 'all': 1,""violet's"": 2, 'bring': ...",1.0
Vulli Sophie the GiraffeTeether ...,I'm not sure why Sophieis such a hit with the ...,4.0,"{'adoring': 1, 'find': 1,'month': 1, 'bright': 1, ...",0.999999999703
Vulli Sophie the GiraffeTeether ...,I'll be honest...I boughtthis toy because all the ...,4.0,"{'all': 2, 'discovered':1, 'existence.': 1, ...",0.999999999392
Vulli Sophie the GiraffeTeether ...,We got this littlegiraffe as a gift from a ...,5.0,"{'all': 2, ""don't"": 1,'(literally).so': 1, ...",0.99999999919
Vulli Sophie the GiraffeTeether ...,As a mother of 16monthold twins; I bought ...,5.0,"{'cute': 1, 'all': 1,'reviews.': 2, 'just' ...",0.999999998657
Vulli Sophie the GiraffeTeether ...,Sophie the Giraffe is theperfect teething toy. ...,5.0,"{'just': 2, 'both': 1,'month': 1, 'ears,': 1, ...",0.999999997108
Vulli Sophie the GiraffeTeether ...,Sophie la giraffe isabsolutely the best toy ...,5.0,"{'and': 5, 'the': 1,'all': 1, 'that': 2, ...",0.999999995589
Vulli Sophie the GiraffeTeether ...,My 5-mos old son took tothis immediately. The ...,5.0,"{'just': 1, 'shape': 2,'mutt': 1, '""dog': 1, ...",0.999999995573
Vulli Sophie the GiraffeTeether ...,My nephews and my fourkids all had Sophie in ...,5.0,"{'and': 4, 'chew': 1,'all': 1, 'perfect;': 1, ...",0.999999989527
Vulli Sophie the GiraffeTeether ...,Never thought I'd see myson French kissing a ...,5.0,"{'giggles': 1, 'all': 1,'out,': 1, 'over': 1, ...",0.999999985069


### 가장 긍정적인 리뷰 확인하기

가장 긍정적인 리뷰를 출력해 봅시다. 어떻게 해야 할까요?

참고: 먼저 가장 긍정적인 리뷰가 속한 줄을 찾아 내고 그 다음에 review 컬럼을 출력해 보세요.

In [38]:
giraffe_reviews[0]['review']

"Sophie, oh Sophie, your time has come. My granddaughter, Violet is 5 months old and starting to teeth. What joy little Sophie brings to Violet. Sophie is made of a very pliable rubber that is sturdy but not tough. It is quite easy for Violet to twist Sophie into unheard of positions to get Sophie into her mouth. The little nose and hooves fit perfectly into small mouths, and the drooling has purpose. The paint on Sophie is food quality.Sophie was born in 1961 in France. The maker had wondered why there was nothing available for babies and made Sophie from the finest rubber, phthalate-free on St Sophie's Day, thus the name was born. Since that time millions of Sophie's populate the world. She is soft and for babies little hands easy to grasp. Violet especially loves the bumpy head and horns of Sophie. Sophie has a long neck that easy to grasp and twist. She has lovely, sizable spots that attract Violet's attention. Sophie has happy little squeaks that bring squeals of delight from Viol

그 다음으로 긍정적인 리뷰를 확인해 봅시다.

In [39]:
giraffe_reviews[1]['review']

"I'm not sure why Sophie is such a hit with the little ones, but my 7 month old baby girl is one of her adoring fans.  The rubber is softer and more pleasant to handle, and my daughter has enjoyed chewing on her legs and the nubs on her head even before she started teething.  She also loves the squeak that Sophie makes when you squeeze her.  Not sure what it is but if Sophie is amongst a pile of her other toys, my daughter will more often than not reach for Sophie.  And I have the peace of mind of knowing that only edible and safe paints and materials have been used to make Sophie, as opposed to Bright Starts and other baby toys made in China.  Now that the research is out on phthalates and other toxic substances in baby toys, I think it's more important than ever to find good quality toys that are also safe for our babies to handle and put in their mouths.  Sophie is a must-have for every new mom in my opinion.  Even if your kid is one of the few that can take or leave her, it's worth

### 가장 부정적인 리뷰를 확인하기

In [40]:
giraffe_reviews[-1]['review']

"My son (now 2.5) LOVED his Sophie, and I bought one for every baby shower I've gone to. Now, my daughter (6 months) just today nearly choked on it and I will never give it to her again. Had I not been within hearing range it could have been fatal. The strange sound she was making caught my attention and when I went to her and found the front curved leg shoved well down her throat and her face a purply/blue I panicked. I pulled it out and she vomited all over the carpet before screaming her head off. I can't believe how my opinion of this toy has changed from a must-have to a must-not-use. Please don't disregard any of the choking hazard comments, they are not over exaggerated!"

두번째로 부정적인 리뷰를 출력해 봅시다.

In [41]:
giraffe_reviews[-2]['review']

"This children's toy is nostalgic and very cute. However, there is a distinct rubber smell and a very odd taste, yes I tried it, that my baby did not enjoy. Also, if it is soiled it is extremely difficult to clean as the rubber is a kind of porus material and does not clean well. The final thing is the squeaking device inside which stopped working after the first couple of days. I returned this item feeling I had overpaid for a toy that was defective and did not meet my expectations. Please do not be swayed by the cute packaging and hype surounding it as I was. One more thing, I was given a full refund from Amazon without any problem."

### 새로운 Sentiment 모델 만들기

**실무에서는 종종 중요하지 않은 단어들을 제거한 후 모델을 학습시킵니다. 이 방법을 이용하면 정확도를 높일 수 있습니다.**

이하에서는 몇몇 단어들만을 이용해서 학습을 시켜보겠습니다. 이 경우 단어수가 너무 작아서 정확도는 낮아지겠지만 classifier가 무슨 일을 하는지를 이해하는데 도움이 될 겁니다.

먼저 다음 단어를 이용하여 **selected_words**라는 list를 만듭니다.

    'awesome', 'great', 'fantastic', 'amazing', 'love', 'horrible', 'bad', 'terrible', 'awful', 'wow', 'hate'

In [42]:
selected_words = ['awesome','great','fantastic','amazing','love','horrible','bad','terrible','awful','wow','hate']

각 단어들이 리뷰에서 사용된 횟수를 센 다음 해당 단어 이름의 컬럼을 만들어서 저장하고자 합니다. 즉, 단어마다 새로운 컬럼을 만들어야 합니다. 예를 들어 'awesome'에 대한 컬럼은 아래와 같이 지정할 수 있습니다.

    filtered_products['awesome']
    
이 방법을 이용하여 모든 단어에 대한 word_count가 아닌, **특정 단어들만의 word_count**를 이용해서 모델을 학습시킬 겁니다. 

for-loop를 이용해서 리스트에 속한 각 단어들을 출력해 보세요.

    for 변수 in 리스트:
        반복할 명령어

In [43]:
for i in selected_words:
    print i

awesome
great
fantastic
amazing
love
horrible
bad
terrible
awful
wow
hate


dict라는 dictionary에 있는 특정 필드 (예. awesome)의 값을 아래와 같은 방법으로 구할 수 있습니다.

    dict['awesome']
    
주의: 만약 dict에 awesome이 없다면 에러가 납니다. 따라서 조건문을 이용해서 해당 단어의 키가 dictionary에 있는지 없는지를 먼저 확인해야 합니다. 아래의 코드로 확인할 수 있습니다.

    if 'awesome' in dict:

filtered_products['word_count'][0]에 특정 단어 ('love')의 값이 들어 있는지 확인하는 조건문을 만들어 보세요. 만약 해당 단어의 값이 있다면 해당 단어의 개수를 출력하고 없다면 0을 출력하는 코드를 작성
해 봅시다.

In [44]:
if 'love' in filtered_products['word_count'][0]:
    print filtered_products['word_count'][0]['love']
else:
    print 0

1


**1**이 출력되어야 합니다.

자, 이제 위의 코드를 함수로 만들어 봅시다. dictionary에서 특정 단어 (w)를 찾아서 해당 단어의 개수를 반환하는 함수입니다. **selected_word_count**라는 이름의 함수를 만들어 봅시다.

    def selected_word_count(인자):
        실행할 코드

In [45]:
def selected_word_count(dict,w):
    if w in dict:
        return dict[w]
    else:
        return 0

그 다음 위에서 만든 for-loop 안에 이 함수를 넣고 데이터 각 행의 'word_count'에 apply 메소드를 이용해서 함수를 적용(!)하여 단어가 등장하는 횟수를 구하고 해당 단어 이름의 컬럼에 저장하는 코드를 만들어 봅시다.

** 코드의 올바른 작동여부를 확인하기 위해 for-loop가 실행될 때마다 단어와 해당 단어가 등장한 총횟수를 출력합시다. **

In [46]:
for w in selected_words:
    filtered_products[w] = filtered_products['word_count'].apply(selected_word_count(dict,w))
    
    # 각 행에 어떻게 함수를 적용하는 건가요ㅠㅠ

TypeError: argument of type 'type' is not iterable

### (고급) lambda를 이용한 코드

아래는 lambda를 이용한 더 효율적인 코드입니다. apply를 이용할 때 일반함수를 사용하면 2개의 argument를 사용할 수 없습니다. 이 문제를 해결하기 위해 lambda를 사용합니다. 또한 함수를 반복적으로 지정하지 않는 장점이 있습니다.

In [47]:
def selected_word_count_adv(dict, w):
    if w in dict:
        return dict[w]
    else:
        return 0

for w in selected_words:
    filtered_products[w] = filtered_products['word_count'].apply(lambda x:selected_word_count_adv(x, w))
    print w, filtered_products[w].sum()

awesome 2002
great 42420
fantastic 873
amazing 1305
love 40277
horrible 659
bad 3197
terrible 673
awful 345
wow 131
hate 1057


# Quiz

## Quiz 1,2

위의 결과값을 확인하면 quiz 1,2를 풀 수 있습니다.

filtered_products에 새롭게 추가한 컬럼들이 있기 때문에 **new_train_data**, **new_test_data**를 만듭니다.

    SFrame.random_split(비율, seed=시드값)

In [48]:
new_train_data,new_test_data = filtered_products.random_split(.8, seed=0)

새롭게 만든 데이터를 이용해서 모델을 학습시킵시다. **selected_words**를 이용해서 만든 컬럼들을 features로 사용하세요.

    graphlab.logistic_classifier.create(학습데이터, target=예측하고자하는값의컬럼이름, features=feature로사용하고자하는데이터의컬럼이름리스트, validation_set=검증데이터)

이전과 마찬가지로 검증데이터로 new_test_data를 사용하세요.

이렇게 생성한 모델을 **selected_words_model**이라는 변수에 저장하세요.

In [49]:
selected_words_model = graphlab.logistic_classifier.create(new_train_data, target='sentiment', features=selected_words, validation_set=new_test_data)

Logistic regression:

--------------------------------------------------------

Number of examples          : 133448

Number of classes           : 2

Number of feature columns   : 11

...

| 3         | 4        | 0.670707     | 0.844276          | 0.843142            |

| 4         | 5        | 0.859040     | 0.844269          | 0.843142            |

| 5         | 6        | 1.052035     | 0.844269          | 0.843142            |

| 6         | 7        | 1.243817     | 0.844269          | 0.843142            |

+-----------+----------+--------------+-------------------+---------------------+

selected_words_model을 출력해 봅시다.

In [50]:
print selected_words_model

SUCCESS: Optimal solution found.

Class                          : LogisticClassifier

Schema
------
Number of coefficients         : 12
Number of examples             : 133448
Number of classes              : 2
Number of feature columns      : 11
Number of unpacked features    : 11

Hyperparameters
---------------
L1 penalty                     : 0.0
L2 penalty                     : 0.01

Training Summary
----------------
Solver                         : newton
Solver iterations              : 6
Solver status                  : SUCCESS: Optimal solution found.
Training time (sec)            : 1.2887

Settings
--------
Log-likelihood                 : 54057.6401

Highest Positive Coefficients
-----------------------------
love                           : 1.3999
(intercept)                    : 1.3673
awesome                        : 1.058
amazing                        : 0.8928
fantastic                      : 0.8913

Lowest Negative Coefficients
----------------------------
terrible                       : -2.0905
hor

먼저 해당 모델의 coefficients를 출력해 봅시다.

In [51]:
print selected_words_model['coefficients']

+-------------+-------+-------+-----------------+------------------+
|     name    | index | class |      value      |      stderr      |
+-------------+-------+-------+-----------------+------------------+
| (intercept) |  None |   1   |  1.36728315229  | 0.00861805467824 |
|   awesome   |  None |   1   |  1.05800888878  |  0.110865296265  |
|    great    |  None |   1   |  0.883937894898 | 0.0217379527921  |
|  fantastic  |  None |   1   |  0.891303090304 |  0.154532343591  |
|   amazing   |  None |   1   |  0.892802422509 |  0.127989503231  |
|     love    |  None |   1   |  1.39989834302  | 0.0287147460124  |
|   horrible  |  None |   1   |  -1.99651800559 | 0.0973584169028  |
|     bad     |  None |   1   | -0.985827369929 | 0.0433603009142  |
|   terrible  |  None |   1   |  -2.09049998487 | 0.0967241912229  |
|    awful    |  None |   1   |  -1.76469955631 |  0.134679803365  |
+-------------+-------+-------+-----------------+------------------+
[12 rows x 5 columns]
Note: Only t

각 단어가 하나의 feature이기 때문에 각각의 weight값을 가지는 것을 확인할 수 있습니다.

## Quiz 3

가장 큰 양의 weight를 가진 단어를 찾기 위해서 해당값의 내림차순으로 소팅해서 출력해 봅시다. 위에서도 확인할 수 있듯이 selected_words_model['coefficients']는 SFrame 유형입니다.

    SFrame.sort(컬럼이름, ascending=True또는False)
    
weight를 나타내는 컬럼이름은 **'value'**입니다.

In [52]:
selected_words_model['coefficients'].sort('value',ascending=False)

name,index,class,value,stderr
love,None,1,1.39989834302,0.0287147460124
(intercept),None,1,1.36728315229,0.00861805467824
awesome,None,1,1.05800888878,0.110865296265
amazing,None,1,0.892802422509,0.127989503231
fantastic,None,1,0.891303090304,0.154532343591
great,None,1,0.883937894898,0.0217379527921
wow,None,1,-0.0541450123332,0.275616449416
bad,None,1,-0.985827369929,0.0433603009142
hate,None,1,-1.40916406276,0.0771983993506
awful,None,1,-1.76469955631,0.134679803365


주의: coefficients는 총 12개 (intercept + 11개 단어)인데 화면에 출력되는 개수는 10개라는 것을 확인할 수 있습니다. 일반적인 print 명령을 이용하면 최대 10개까지 출력됩니다. 따라서 12개 모두를 출력하기 위해서는 아래의 메소드를 이용해야 합니다.

    SFrame.print_rows(num_rows=12)

In [53]:
selected_words_model['coefficients'].sort('value',ascending=False).print_rows(num_rows=12)

+-------------+-------+-------+------------------+------------------+
|     name    | index | class |      value       |      stderr      |
+-------------+-------+-------+------------------+------------------+
|     love    |  None |   1   |  1.39989834302   | 0.0287147460124  |
| (intercept) |  None |   1   |  1.36728315229   | 0.00861805467824 |
|   awesome   |  None |   1   |  1.05800888878   |  0.110865296265  |
|   amazing   |  None |   1   |  0.892802422509  |  0.127989503231  |
|  fantastic  |  None |   1   |  0.891303090304  |  0.154532343591  |
|    great    |  None |   1   |  0.883937894898  | 0.0217379527921  |
|     wow     |  None |   1   | -0.0541450123332 |  0.275616449416  |
|     bad     |  None |   1   | -0.985827369929  | 0.0433603009142  |
|     hate    |  None |   1   |  -1.40916406276  | 0.0771983993506  |
|    awful    |  None |   1   |  -1.76469955631  |  0.134679803365  |
|   horrible  |  None |   1   |  -1.99651800559  | 0.0973584169028  |
|   terrible  |  Non

어떤 단어가 가장 큰 양의 weight값을 가지고 있나요?

## Quiz 4

가장 큰 음의 weight를 가진 단어를 찾기 위해서 올림차순으로 출력해 봅시다.

In [54]:
selected_words_model['coefficients'].sort('value',ascending=True).print_rows(num_rows=12)

+-------------+-------+-------+------------------+------------------+
|     name    | index | class |      value       |      stderr      |
+-------------+-------+-------+------------------+------------------+
|   terrible  |  None |   1   |  -2.09049998487  | 0.0967241912229  |
|   horrible  |  None |   1   |  -1.99651800559  | 0.0973584169028  |
|    awful    |  None |   1   |  -1.76469955631  |  0.134679803365  |
|     hate    |  None |   1   |  -1.40916406276  | 0.0771983993506  |
|     bad     |  None |   1   | -0.985827369929  | 0.0433603009142  |
|     wow     |  None |   1   | -0.0541450123332 |  0.275616449416  |
|    great    |  None |   1   |  0.883937894898  | 0.0217379527921  |
|  fantastic  |  None |   1   |  0.891303090304  |  0.154532343591  |
|   amazing   |  None |   1   |  0.892802422509  |  0.127989503231  |
|   awesome   |  None |   1   |  1.05800888878   |  0.110865296265  |
| (intercept) |  None |   1   |  1.36728315229   | 0.00861805467824 |
|     love    |  Non

어떤 단어가 가장 큰 음의 weight값을 가지고 있나요?

## 3개의 분석 모델 간의 정확도 비교하기

이제 3개의 모델을 이용해서 각 모델 간의 정확도를 비교해 보겠습니다.

selected_words_model, sentiment_model, majority class 등 3가지 모델을 사용하고자 합니다.

## Quiz 5

먼저 새로이 만든 모델 (selected_words_model)을 **new_test_data**를 이용하여 평가해 봅시다. (반드시 new_test_data를 사용해야 합니다. 왜일까요?)

    model.evaluate(데이터)
    
맨 윗줄에서 accuracy를 확인할 수 있습니다. 아래의 내용을 자세히 살펴 보세요.

In [55]:
selected_words_model.evaluate(new_test_data)

{'accuracy': 0.8431419649291376,
 'auc': 0.6648096413721418,
 'confusion_matrix': Columns:
 	target_label	int
 	predicted_label	int
 	count	int
 
 Rows: 4
 
 Data:
 +--------------+-----------------+-------+
 | target_label | predicted_label | count |
 +--------------+-----------------+-------+
 |      0       |        0        |  234  |
 |      0       |        1        |  5094 |
 |      1       |        1        | 27846 |
 |      1       |        0        |  130  |
 +--------------+-----------------+-------+
 [4 rows x 3 columns],
 'f1_score': 0.914242563530107,
 'log_loss': 0.4054747110365535,
 'precision': 0.8453551912568306,
 'recall': 0.9953531598513011,
 'roc_curve': Columns:
 	threshold	float
 	fpr	float
 	tpr	float
 	p	int
 	n	int
 
 Rows: 100001
 
 Data:
 +-----------+-----+-----+-------+------+
 | threshold | fpr | tpr |   p   |  n   |
 +-----------+-----+-----+-------+------+
 |    0.0    | 1.0 | 1.0 | 27976 | 5328 |
 |   1e-05   | 1.0 | 1.0 | 27976 | 5328 |
 |   2e-05   | 

selected_words_model의 accuracy는 얼마인가요?

## Quiz 6

이전에 만든 sentiment_model을 **test_data**로 평가해 봅시다.
역시 맨 윗줄에서 accuracy를 확인할 수 있습니다.

    model.evaluate(데이터)

In [56]:
sentiment_model.evaluate(test_data)

{'accuracy': 0.916256305548883,
 'auc': 0.9446492867438502,
 'confusion_matrix': Columns:
 	target_label	int
 	predicted_label	int
 	count	int
 
 Rows: 4
 
 Data:
 +--------------+-----------------+-------+
 | target_label | predicted_label | count |
 +--------------+-----------------+-------+
 |      0       |        1        |  1328 |
 |      0       |        0        |  4000 |
 |      1       |        1        | 26515 |
 |      1       |        0        |  1461 |
 +--------------+-----------------+-------+
 [4 rows x 3 columns],
 'f1_score': 0.9500349343413533,
 'log_loss': 0.2610669843242232,
 'precision': 0.9523039902309378,
 'recall': 0.9477766657134686,
 'roc_curve': Columns:
 	threshold	float
 	fpr	float
 	tpr	float
 	p	int
 	n	int
 
 Rows: 100001
 
 Data:
 +-----------+----------------+----------------+-------+------+
 | threshold |      fpr       |      tpr       |   p   |  n   |
 +-----------+----------------+----------------+-------+------+
 |    0.0    |      1.0       |  

sentiment_model의 accuracy는 얼마인가요?

## Quiz 7

Majority class prediction은 test_data에서 가장 많은 sentiment (1 또는 0)를 선택해서 모든 데이터에 대해 예측하는 것을 말합니다. 모델을 평가할 때 baseline으로 삼을 수 있습니다.

test_data의 sentiment를 분류형 ('Categorical')으로 출력해 보세요.

    SArray.show(view=뷰유형)

In [57]:
test_data['sentiment'].show(view='Categorical')

어떤 값이 가장 많은가요? 이 경우 Accuracy는 얼마일까요?

이 경우 sentiment 값이 0또는 1이기 때문에 단순히 sentiment를 더하는 것만으로도 majority class를 구할 수 있습니다. **왜일까요?**

Accuracy는 맞게 예측한 값을 전체 개수로 나누면 구할 수 있습니다.

주의: 나누기할 때 반드시 float()를 적용해야만 소수점 계산이 가능합니다. 피제수(dividend) 또는 제수(divisor)의 유형을 float로 변환하여야 합니다.

In [58]:
float(len(test_data[test_data['sentiment']==1]))/float(len(test_data))

0.8400192169108815

## Quiz 8

위의 3가지 모델의 정확도(accuracy)를 비교해 보세요.



## Quiz 9

모델 간의 결과가 왜 이렇게 다른 것일까요? 특정 상품의 리뷰를 이용해서 왜 그런지 알아 봅시다.

sentiment_model을 이용해서 'Baby Trend Diaper Champ'라는 상품 리뷰의 predicted_sentiment를 구하고 그 중에서 가장 긍정적인 predicted_sentiment를 가진 리뷰를 찾아 봅시다.

먼저 filtered_products에서 해당 상품의 리뷰만을 골라냅니다.

In [62]:
diaper_champ_reviews = filtered_products[filtered_products['name']=='Baby Trend Diaper Champ']

먼저 몇 개인지 확인하세요. 298개가 맞나요?

In [63]:
len(diaper_champ_reviews)

298

이제 sentiment_model을 이용해서 각 리뷰들의 예측값을 구한 다음 **predicted_sentiment** 컬럼에 저장하세요.

    model.predict(데이터, output_type=출력유형)
    
출력유형으로 'probability'를 지정합시다.

In [64]:
diaper_champ_reviews['predicted_sentiment'] = sentiment_model.predict(diaper_champ_reviews, output_type='probability')

diaper_champ_reviews를 출력해서 잘 저장되었는지 확인해 봅시다.

In [65]:
print diaper_champ_reviews

+-------------------------+-------------------------------+--------+
|           name          |             review            | rating |
+-------------------------+-------------------------------+--------+
| Baby Trend Diaper Champ | Ok - newsflash.  Diapers a... |  4.0   |
| Baby Trend Diaper Champ | My husband and I selected ... |  1.0   |
| Baby Trend Diaper Champ | Excellent diaper disposal ... |  5.0   |
| Baby Trend Diaper Champ | We love our diaper champ. ... |  5.0   |
| Baby Trend Diaper Champ | Two girlfriends and two fa... |  5.0   |
| Baby Trend Diaper Champ | I waited to review this un... |  4.0   |
| Baby Trend Diaper Champ | I have had a diaper genie ... |  1.0   |
| Baby Trend Diaper Champ | I originally put this item... |  5.0   |
| Baby Trend Diaper Champ | I am so glad I got the Dia... |  5.0   |
| Baby Trend Diaper Champ | We had 2 diaper Genie's bo... |  4.0   |
+-------------------------+-------------------------------+--------+
+-------------------------------+-

이제 predicted_sentiment를 기준으로 내림차순 소팅을 해서 가장 높은 predicted_sentiment값을 가진 리뷰를 찾아 봅시다.

In [66]:
diaper_champ_reviews.sort('predicted_sentiment',ascending=False)

name,review,rating,word_count,sentiment,awesome
Baby Trend Diaper Champ,Baby Luke can turn aclean diaper to a dirty ...,5.0,"{'all': 1, 'less': 1,""friend's"": 1, '(which': ...",1,0
Baby Trend Diaper Champ,I LOOOVE this diaperpail! Its the easies ...,5.0,"{'just': 1, 'over': 1,'rweek': 1, 'sooo': 1, ...",1,0
Baby Trend Diaper Champ,We researched all of thedifferent types of di ...,4.0,"{'all': 2, 'just': 4,""don't"": 2, 'one,': 1, ...",1,0
Baby Trend Diaper Champ,My baby is now 8 monthsand the can has been ...,5.0,"{""don't"": 1, 'able': 2,'over': 1, 'soon': 1, ...",1,0
Baby Trend Diaper Champ,"This is absolutely, byfar, the best diaper ...",5.0,"{'just': 3, 'money': 1,'still': 3, 'fine': 1, ...",1,0
Baby Trend Diaper Champ,Diaper Champ or DiaperGenie? That was my ...,5.0,"{'son': 2, 'all': 1,'bags.': 1, 'son,': 1, ...",1,0
Baby Trend Diaper Champ,Wow! This is fabulous.It was a toss-up between ...,5.0,"{'and': 4, 'this': 3,'stink': 1, 'garbage' ...",1,0
Baby Trend Diaper Champ,I originally put thisitem on my baby registry ...,5.0,"{'lysol': 1, 'all': 2,'bags.': 1, 'feedback': ...",1,0
Baby Trend Diaper Champ,Two girlfriends and twofamily members put me ...,5.0,"{'just': 1, '-': 3,'both': 1, 'results': 1, ...",1,0
Baby Trend Diaper Champ,I am one of those super-critical shoppers who ...,5.0,"{'all': 1, 'humid': 1,'just': 1, 'less': 1, ...",1,0


아래에서 계속 사용하기 위해 이렇게 소팅한 대로 다시 diaper_champ_reviews에 저장합시다.

참고: 쉽게 순서를 변경할 수 있는 소팅의 경우 변수를 재활용하기도 합니다.

In [67]:
diaper_champ_reviews = diaper_champ_reviews.sort('predicted_sentiment',ascending=False)

### slice notation을 이용해서 SFrame의 특정 데이터를 선택하기

퀴즈9를 풀기 전에 먼저 데이터를 선택해서 slice notation을 이용해서 SFrame의 특정 데이터를 선택하는 방법에 대해서 알아 봅시다.

**products의 맨 첫번째 줄만**을 출력해 봅시다.

In [68]:
print products[0]

{'rating': 3.0, 'review': 'These flannel wipes are OK, but in my opinion not worth keeping.  I also ordered someImse Vimse Cloth Wipes-Ocean Blue-12 countwhich are larger, had a nicer, softer texture and just seemed higher quality.  I use cloth wipes for hands and faces and have been usingThirsties 6 Pack Fab Wipes, Boyfor about 8 months now and need to replace them because they are starting to get rough and have had stink issues for a while that stripping no longer handles.', 'name': 'Planetwise Flannel Wipes', 'word_count': {'and': 5, '6': 1, 'stink': 1, 'because': 1, 'ordered': 1, 'just': 1, 'boyfor': 1, 'wipes-ocean': 1, 'wipes,': 1, 'replace': 1, 'not': 1, 'softer': 1, 'are': 3, 'have': 2, 'in': 1, 'need': 1, 'rough': 1, 'ok,': 1, 'issues': 1, 'seemed': 1, 'use': 1, 'blue-12': 1, 'vimse': 1, 'for': 2, 'no': 1, 'that': 1, 'larger,': 1, 'been': 1, 'to': 2, 'someimse': 1, 'quality.': 1, '8': 1, 'flannel': 1, 'worth': 1, 'higher': 1, 'them': 1, 'get': 1, 'keeping.': 1, 'countwhich': 1

type 명령을 이용해서 유형을 확인합시다.

In [69]:
type(products[0])

dict

SFrame에서 하나의 줄만을 선택한 경우 dict 유형임을 확인할 수 있습니다.

이제 slice notation을 이용해서 첫번째 줄만을 선택해 봅시다.

In [70]:
products[0:1]

name,review,rating,word_count
Planetwise Flannel Wipes,"These flannel wipes areOK, but in my opinion ...",3.0,"{'and': 5, '6': 1,'stink': 1, 'because' ..."


type 명령을 이용해서 유형을 확인해 봅시다.

In [71]:
type(products[0:1])

graphlab.data_structures.sframe.SFrame

slice notation을 이용해서 데이터를 선택하면 SFrame 유형임을 확인할 수 있습니다.

### 가장 긍정적인 리뷰를 확인하기

자, 이제 다시 돌아가서 diaper_champ_reviews를 분석해 봅시다. 이 리뷰 중에서 가장 긍정적인 리뷰의 predicted_sentiment를 출력해 봅시다.

주의: model.predict에서는 SFrame을 인자로 받기 때문에 SFrame을 사용해야 합니다.

In [72]:
diaper_champ_reviews[0:1]['predicted_sentiment']

dtype: float
Rows: 1
[0.9999999372669541]

실제 리뷰 내용을 확인해 봅시다.

In [73]:
diaper_champ_reviews[0:1]['review']

dtype: str
Rows: 1
['Baby Luke can turn a clean diaper to a dirty diaper in 3 seconds flat. The diaper champ turns the smelly diaper into "what diaper smell" in less time than that. I hesitated and wondered what I REALLY needed for the nursery. This is one of the best purchases we made. The champ, the baby bjorn, fluerville diaper bag, and graco pack and play bassinet all vie for the best baby purchase.Great product, easy to use, economical, effective, absolutly fabulous.UpdateI knew that I loved the champ, and useing the diaper genie at a friend's house REALLY reinforced that!! There is no comparison, the chanp is easy and smell free, the genie was difficult to use one handed (which is absolutly vital if you have a little one on a changing pad) and there was a deffinite odor eminating from the genieplus we found that the quick tie garbage bags where the ties are integrated into the bag work really well because there isn't any added bulk around the sealing edge of the champ.']

## Quiz 10

selected_words_model을 이용해서 해당 리뷰 (가장 긍정적인 리뷰)의 예측값을 구해 봅시다.

    model.predict(데이터, output_type=출력유형)
    
출력유형으로 'probability'를 지정합시다.

주의: model.predict에서는 데이터로 SFrame을 인자로 받습니다.

In [74]:
selected_words_model.predict(diaper_champ_reviews[0:1],output_type='probability')

dtype: float
Rows: 1
[0.7969408512906639]

## Quiz 11

해당 리뷰의 모든 데이터를 출력해 봅시다.

In [76]:
diaper_champ_reviews[0:1].show()

데이터를 살펴 보면 selected_words에 속한 단어들이 단 한번도 나오지 않는다는 것을 확인할 수 있습니다.